<a href="https://colab.research.google.com/github/i475yama/test/blob/main/Battacharrya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# OpenCV C++ の開発環境をインストール（初回のみ）
!apt update
!apt install -y libopencv-dev g++ cmake

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,665 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,926 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,245 kB]
Hit:13 https://ppa.launchp

In [7]:
%%writefile Battacharrya.cpp
#include <cmath>
#include <opencv2/opencv.hpp>
#include <iostream>

// 定数
const int QSTEP = 16;
const int QSIZE = 256 / QSTEP;
const int NBIN = QSIZE * QSIZE * QSIZE;

void set_kernel(cv::Mat_<float> &kernel, cv::Size ksize) {
    kernel.create(ksize);
    int w = ksize.width;
    int h = ksize.height;
    for (int y = 0; y < h; ++y)
        for (int x = 0; x < w; ++x) {
            float dx = (x - w / 2.0f) / (w / 2.0f);
            float dy = (y - h / 2.0f) / (h / 2.0f);
            kernel(y, x) = std::exp(-(dx * dx + dy * dy));
        }
}

void calc_rgb_histogram(cv::Mat_<float> &hist, const cv::Mat_<cv::Vec3b> &rgb, const cv::Mat_<float> &kernel) {
    hist = cv::Mat_<float>::zeros(NBIN, 1);
    for (int y = 0; y < rgb.rows; ++y) {
        for (int x = 0; x < rgb.cols; ++x) {
            cv::Vec3b color = rgb(y, x);
            int r = color[2] / QSTEP;
            int g = color[1] / QSTEP;
            int b = color[0] / QSTEP;
            int bin = (r * QSIZE + g) * QSIZE + b;
            hist(bin) += kernel(y, x);
        }
    }
    // 正規化
    float sum = cv::sum(hist)[0];
    if (sum > 0) hist /= sum;
}

float calc_bhattacharry(const cv::Mat_<float> &hist1, const cv::Mat_<float> &hist2) {
    return cv::sum(hist1.mul(hist2))[0];
}

int main(int argc, char *argv[]) {
    if (argc < 2) {
        std::cerr << "Usage: ./a.out image.jpg" << std::endl;
        return 1;
    }

    cv::Mat_<cv::Vec3b> image = cv::imread(argv[1], cv::IMREAD_COLOR);
    if (image.empty()) {
        std::cerr << "Image load error" << std::endl;
        return 1;
    }

    cv::Rect target(359, 175, 67, 99);
    cv::Mat_<float> kernel;
    set_kernel(kernel, target.size());

    cv::Mat_<float> model(NBIN, 1);
    calc_rgb_histogram(model, image(target), kernel);

    cv::Mat_<float> similarity_map = cv::Mat_<float>::zeros(image.size());

    for (int y = 0; y <= image.rows - target.height; ++y) {
        for (int x = 0; x <= image.cols - target.width; ++x) {
            cv::Rect r(x, y, target.width, target.height);
            cv::Mat_<float> cand(NBIN, 1);
            calc_rgb_histogram(cand, image(r), kernel);
            float sim = calc_bhattacharry(model, cand);
            int cx = x + target.width / 2;
            int cy = y + target.height / 2;
            similarity_map(cy, cx) = sim;
        }
    }

    cv::normalize(similarity_map, similarity_map, 0, 255, cv::NORM_MINMAX);
    similarity_map.convertTo(similarity_map, CV_8U);
    cv::imwrite("similarity.png", similarity_map);
    return 0;
}


Writing Battacharrya.cpp


In [8]:
!g++ Battacharrya.cpp -o Battacharrya `pkg-config --cflags --libs opencv4`

In [9]:
from google.colab import files
uploaded = files.upload()

Saving MVI_0182-0000300.png to MVI_0182-0000300.png


In [14]:
!./Battacharrya MVI_0182-0000300.png

terminate called after throwing an instance of 'cv::Exception'
  what():  OpenCV(4.5.4) ./modules/core/src/matrix_wrap.cpp:1166: error: (-215:Assertion failed) !fixedType() || ((Mat*)obj)->type() == mtype in function 'create'



In [15]:
from IPython.display import Image
Image(filename='')

FileNotFoundError: [Errno 2] No such file or directory: 'similarity.png'